Copyright (c) 2022, Jash Vira
All rights reserved.

This source code is licensed under the BSD-style license found in the
LICENSE file in the root directory of this source tree. 

In [15]:
import numpy as np
import cvxpy as cp
import cvxopt
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

Splitting data into train and validation.
\
Test data is provided seperately.
\
Regularisation parameter *c* is set to 100.

In [16]:
data = np.loadtxt('train.csv', delimiter=',')
training_data = data[0:4000, :]
y_train = training_data[:, 0]
y_train = (y_train *2)-1
x_train = training_data[:,1:]

validation_data = data[-4500:, :]
y_validation = validation_data[:, 0]
y_validation = (y_validation *2)-1
x_validation = validation_data[:,1:]

testing_data = np.loadtxt('test.csv', delimiter=',')
y_test = testing_data[:, 0]
y_test = (y_test *2)-1
x_test = testing_data[:,1:]
c = 100


Algorithm for primal form using CVXPY.

In [17]:
def svm_predict_primal(test_data, test_labels, model):
  w = model[0]
  intercept = model[1]  
  correct = 0
  for i in range(len(test_data)):
    if test_labels[i] == 1:
      if (w.T).dot(test_data[i]) + intercept > 0:
        correct += 1
    else:
      if (w.T).dot(test_data[i]) + intercept <= 0:
        correct += 1
  return correct/len(test_data)

def svm_train_primal(data_train , label_train , regularisation_para_C):
  x = data_train
  y = label_train
  c = regularisation_para_C
  m,n = x.shape
  W = cp.Variable((n,1))
  Psi = cp.Variable((m,1))
  b = cp.Variable()

  # Solving for primal problem
  objective_primal = cp.Minimize((1/2)*cp.square(cp.norm(W))+ ((c/m)* cp.sum(Psi)))
  constraints_primal=[]
  for i in range(m):
    constraints_primal.extend([y[i]*(W.T@x[i]+b)+Psi[i]>=1,Psi[i]>=0])

  prob = cp.Problem(objective_primal, constraints_primal)
  prob.solve()

  w_values = np.array(W.value)
  b_value = b.value
  Psi = np.array(Psi.value)
  sup_vec_index = []
  threshold = 1e-4

  # Obtaining support vectors, collab RAM crashes if np.where() is used.
  for i in range(m):
    predicted_values = np.dot(w_values.T,x[i])+b_value
    if ((y[i]*predicted_values)-1+Psi[i] < threshold):
        sup_vec_index.append(i)

  primal_support_vec = x[sup_vec_index]
  return (w_values.flatten(), b_value), len(primal_support_vec)

# Training and prediction
model, len_of_primal_support_vec = svm_train_primal(x_train,y_train,c)
ans = svm_predict_primal(x_test,y_test,model)

# Printing results
print("Accuracy: "+ str(ans))
print("Value of b: "+ str(model[1]))
print("Number of support vectors: "+ str(len_of_primal_support_vec))
print("Sum of w: "+ str(np.sum(model[0])))

Accuracy: 0.968
Value of b: 1.7798137171000277
Number of support vectors: 392
Sum of w: -0.1452156803626381


Algorithm for dual form using CVXOPT.

In [18]:
def svm_predict_dual(x,y,w,b):
  correct = 0
  length = x.shape[0]
  for i in range(length):
    y_pred = np.sign(np.dot(np.array(x[i]), w)+b)[0]
    if y_pred == y[i]:
      correct+=1
  return correct/length

def svm_train_dual(X, y, slack):
  m,n = X.shape
  y = y.reshape(-1, 1) * 1
  H = (np.outer(y,y)* np.dot(X,X.T))
  P = cvxopt.matrix(H)
  q = cvxopt.matrix(-np.ones((m,1)))
  tmp1 = np.eye(m) * -1
  tmp2 = np.eye(m)
  G = cvxopt.matrix(np.concatenate((tmp1, tmp2),axis=0))
  tmp1 = np.zeros((m,1))
  tmp2 = np.ones((m,1)) * slack/m
  h = cvxopt.matrix(np.concatenate((tmp1, tmp2),axis=0))
  A = cvxopt.matrix(y.reshape(1, -1))
  b = cvxopt.matrix(0.0)
  
  # Solving dual form.
  cvxopt.solvers.options['show_progress'] = False
  sol = cvxopt.solvers.qp(P, q, G, h, A, b)
  alphas = np.array(sol['x'])
  threshold = 1e-4
  
  # Obtaining w,b and support vectors from alpha.
  w = ((y * alphas).T @ X).reshape(-1,1)
  support_vecs = np.where(((alphas <= slack/m) & (alphas >= threshold)))[0]
  b = y[support_vecs] - np.dot(X[support_vecs], w)
  
  return w, np.average(b), support_vecs, np.sum(alphas)

# Training and prediction
w_dual, b_dual, support_vectors, alphas_sum = svm_train_dual(x_train,y_train,c)
ans = svm_predict_dual(x_test,y_test,w_dual, b_dual)

# Printing results
print("Accuracy: " + str(ans))
print("Average of b: " + str(b_dual))
print("Number of support vectors: " + str(len(support_vectors)))
print("Sum of alphas: " + str(alphas_sum))
print("Sum of w: " + str(np.sum(w_dual)))

Accuracy: 0.9686666666666667
Average of b: 1.7389327654668092
Number of support vectors: 392
Sum of alphas: 7.281637057117689
Sum of w: -0.1451352273951918


Primal form is used to find the optimal *c* value.
\
*c* values tried are in the `best_c` array.

In [20]:
best_c = np.arange(-10,11,2)
accuracy_values = []
for i in best_c:
  model, len_of_primal_support_vec = svm_train_primal(x_train,y_train,2.0**i)
  accuracy_values.append(svm_predict_primal(x_validation, y_validation, model))

print(2.0**(best_c[np.argmax(accuracy_values)]))

4.0


Scikit-learn SVM used with the optimal *c* value found in the step above.



In [21]:
svc = LinearSVC(C=4, dual=False, tol=1e-4)
svc.fit(x_train, y_train)

y_pred = svc.predict(x_test)
print("Accuracy: " + str(svc.score(x_test,y_test)))

Accuracy: 0.9713333333333334
